# Butterfly Analyses
## Dependencies
First we will load the necessary packages.

In [11]:
# Import packages.
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm

Next we define a function to calculate site patterns from derived allele frequency arrays.

In [12]:
def butterfly_site_patterns(
    p1,
    p2,
    p3,
):
    """
    ###########################################################################
    INPUT
        p1: Derived alle frequency array for the P1 lineage.
        p2: Derived alle frequency array for the P2 lineage.
        p3: Derived alle frequency array for the P3 lineage.
    ---------------------------------------------------------------------------
    OUTPUT: Genome counts of ABBA, BABA, BBAA, BAAA, ABAA, and AABA sites.
    ###########################################################################
    """
    # Calculate site pattern counts.
    abba = ((1 - p1) * (p2) * (p3)).sum()
    baba = ((p1) * (1 - p2) * (p3)).sum()
    baaa = ((p1) * (1 - p2) * (1 - p3)).sum()
    abaa = ((1 - p1) * (p2) * (1 - p3)).sum()
    return abba, baba, baaa, abaa

Next we define a function to calculate all the introgression metrics for detecting introgression.

In [13]:
def butterfly_detection(
    abba,
    baba,
    baaa,
    abaa,
):
    """
    ###########################################################################
    INPUT: Genome-wide ABBA, BABA, BAAA, and ABAA counts from frequencies.
    ---------------------------------------------------------------------------
    OUTPUT: Patterson's D, Danc, and D+ values.
    ###########################################################################
    """
    # Calculate Patterson's D.
    d = ((abba - baba) / (abba + baba))
    # Calculate Danc.
    danc = ((baaa - abaa) / (baaa + abaa))
    # Calculate D+.
    dplus = (((abba - baba) + (baaa - abaa)) / ((abba + baba) + (baaa + abaa)))
    return d, danc, dplus

Next we define a function to calculate all the introgression metrics for quantifying introgression.

In [14]:
def butterfly_quantification(
    abba_num,
    baba_num,
    baaa_num,
    abaa_num,
    abba_den,
    baba_den,
    baaa_den,
    abaa_den,
):
    """
    ###########################################################################
    INPUT
        {site pattern}_num: Site pattern counts from ((P1, P2), P3).
        {site pattern}_den: Site pattern counts from ((P1, P3), P3).
    ---------------------------------------------------------------------------
    OUTPUT: fhom, fanc, and f+ values.
    ###########################################################################
    """
    # Calculate fhom.
    fhom = ((abba_num - baba_num) / (abba_den - baba_den))
    # Calculate fanc.
    fanc = ((baaa_num - abaa_num) / (baaa_den - abaa_den))
    # Calculate f+.
    fplus = (((abba_num - baba_num) + (baaa_num - abaa_num)) / ((abba_den - baba_den) + (baaa_den - abaa_den)))
    return fhom, fanc, fplus

Next we define a function standard deviations of each bootstrapped distribution.

In [15]:
def load_bs_stds(
    p1,
    p2,
    p3,
):
    """
    ###########################################################################
    INPUT
        p1: P1 lineage shorthand.
        p2: P2 lineage shorthand.
        p3: P3 lineage shorthand.
    ---------------------------------------------------------------------------
    OUTPUT: Dictionary of standard deviations per bootstrapped distribution.
    ###########################################################################
    """
    # Define the file path for the results.
    results_path = './bootstraps/{0}_{1}_{2}/'.format(p1, p2, p3)
    # Load the bootstrapped distributions.
    bs_abba = np.loadtxt(
        results_path+'abba.csv.gz',
        delimiter=',',
    )
    bs_baba = np.loadtxt(
        results_path+'baba.csv.gz',
        delimiter=',',
    )
    bs_baaa = np.loadtxt(
        results_path+'baaa.csv.gz',
        delimiter=',',
    )
    bs_abaa = np.loadtxt(
        results_path+'abaa.csv.gz',
        delimiter=',',
    )
    bs_abba_hom = np.loadtxt(
        results_path+'abba_hom.csv.gz',
        delimiter=',',
    )
    bs_baba_hom = np.loadtxt(
        results_path+'baba_hom.csv.gz',
        delimiter=',',
    )
    bs_baaa_hom = np.loadtxt(
        results_path+'baaa_hom.csv.gz',
        delimiter=',',
    )
    bs_abaa_hom = np.loadtxt(
        results_path+'abaa_hom.csv.gz',
        delimiter=',',
    )
    # Calculate bootstrapped introgression values.
    bs_d, bs_danc, bs_dplus = butterfly_detection(
        abba=bs_abba,
        baba=bs_baba,
        baaa=bs_baaa,
        abaa=bs_abaa,
    )
    bs_fhom, bs_fanc, bs_fplus = butterfly_quantification(
        abba_num=bs_abba,
        baba_num=bs_baba,
        baaa_num=bs_baaa,
        abaa_num=bs_abaa,
        abba_den=bs_abba_hom,
        baba_den=bs_baba_hom,
        baaa_den=bs_baaa_hom,
        abaa_den=bs_abaa_hom,
    )
    # Create a dictionary where key is the introgression metric and its
    # corresponding numpy array is the value.
    bs_std_dict = {
        'd': np.std(bs_d), 'danc': np.std(bs_danc), 'dplus': np.std(bs_dplus),
        'fhom': np.std(bs_fhom), 'fanc': np.std(bs_fanc), 'fplus': np.std(bs_fplus),
    }
    return bs_std_dict

Define a function to calculate p-values for introgression detection metrics.

In [29]:
def calc_p_values(
    obs_d,
    obs_danc,
    obs_dplus,
    bs_std_d,
    bs_std_danc,
    bs_std_dplus,
):
    """
    ###########################################################################
    INPUT
        obs_{stat}: Observed introgression value.
        bs_std_{stat}: Standard deviation of the bootstrapped distribution.
    ---------------------------------------------------------------------------
    OUTPUT: P-values for introgression detection metrics.
    ###########################################################################
    """
    # Use the survival function to calculate p-values.
    d_pval = norm.sf(x=abs(obs_d), loc=0, scale=abs(bs_std_d))
    danc_pval = norm.sf(x=abs(obs_danc), loc=0, scale=abs(bs_std_danc))
    dplus_pval = norm.sf(x=abs(obs_dplus), loc=0, scale=abs(bs_std_dplus))
    # Create a dictionary where key is the introgression metric and its
    # corresponding numpy array is the value.
    pval_dict = {
        'd': d_pval, 'danc': danc_pval, 'dplus': dplus_pval,
    }
    return pval_dict

Next we load the derived allele frequencies csv into a pandas dataframe.

In [17]:
# Load in the butterfly derived allele frequencies.
der_freq_df = pd.read_csv('./butterflies_wgs_filtered_der_freqs.csv')
der_freq_df

,chromosome,chrom_pos,scaffold,position,chi,chi_1,chi_2,ros,ros_1,ros_2,...,am,am_1,am_2,ag,ag_1,ag_2,timP,timP_1,timP_2,silvaniforms
0,chr1,5609,HE670803,5608,0.125,0.25,0.00,0.000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,0
1,chr1,5613,HE670803,5612,0.125,0.25,0.00,0.000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,0
2,chr1,5642,HE670803,5641,0.125,0.25,0.00,0.000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,0
3,chr1,5643,HE670803,5642,0.000,0.00,0.00,0.875,1.0,0.75,...,1.0,1.0,1.0,1.0,1.0,1.0,0.875,0.75,1.00,0
4,chr1,5664,HE670803,5663,1.000,1.00,1.00,0.125,0.0,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.25,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4783709,chrZ,4120681,HE671531,172429,1.000,1.00,1.00,1.000,1.0,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000,1.00,1.00,0
4783710,chrZ,4120692,HE671531,172440,0.375,0.50,0.25,1.000,1.0,1.00,...,1.0,1.0,1.0,1.0,1.0,1.0,0.000,0.00,0.00,0
4783711,chrZ,4120701,HE671531,172449,0.625,0.50,0.75,0.000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.375,0.50,0.25,0
4783712,chrZ,4120719,HE671531,172467,0.000,0.00,0.00,0.000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,0


Lastly, we extract the genome-wide derived allele frequencies for each population as numpy arrays to make computations easier.

In [18]:
# Extract derived allele frequency arrays.
# H. m. aglaope (aglaope).
aglaope_der_freq_array = der_freq_df['ag'].values
# H. m. amaryllis (amaryllis).
amaryllis_der_freq_array = der_freq_df['am'].values
# H. t. thelxinoe (timareta).
timareta_der_freq_array = der_freq_df['timP'].values
# H. m. melpomene from French Guianna (melp_fg).
melp_fg_der_freq_array = der_freq_df['melFG'].values
# H. m. melpomene from Panama (melp_p).
melp_p_der_freq_array = der_freq_df['melP'].values
# H. m. rosina (rosina).
rosina_der_freq_array = der_freq_df['ros'].values
# H. c. chioneus (cydno).
cydno_der_freq_array = der_freq_df['chi'].values

## Introgression Between  _H. timareta_ & _H. m. amaryllis_
First, we re-examined rates of gene flow between _H. timareta_ and _H. m. amaryllis_ across three time periods: a short, recent period, subsequent to the divergence between _H. m. amaryllis_ and _H. m. aglaope_; an intermediate period, subsequent to the divergence of the Peruvian populations from French Guianan _H. m. melpomene_; and a long period, subsequent to the divergence between Peruvian and Panamanian populations.  
  
First, we will look at the short time period 4—i.e., ((_H. m. aglaope_, _H. m. amaryllis_), _H. timareta_).

In [30]:
# Calculate site pattern counts for.
ag_am_tim_abba, ag_am_tim_baba, ag_am_tim_baaa, ag_am_tim_abaa = butterfly_site_patterns(
    p1=aglaope_der_freq_array,
    p2=amaryllis_der_freq_array,
    p3=timareta_der_freq_array,
)
ag_tim_tim_abba, ag_tim_tim_baba, ag_tim_tim_baaa, ag_tim_tim_abaa = butterfly_site_patterns(
    p1=aglaope_der_freq_array,
    p2=timareta_der_freq_array,
    p3=timareta_der_freq_array,
)
# Calculate site pattern differences.
ag_am_tim_abba_baba = ag_am_tim_abba - ag_am_tim_baba
ag_am_tim_baaa_abaa = ag_am_tim_baaa - ag_am_tim_abaa
# Calculate introgression detection metrics.
ag_am_tim_d, ag_am_tim_danc, ag_am_tim_dplus = butterfly_detection(
    abba=ag_am_tim_abba,
    baba=ag_am_tim_baba,
    baaa=ag_am_tim_baaa,
    abaa=ag_am_tim_abaa,
)
# Calculate introgression quantification metrics.
ag_am_tim_fhom, ag_am_tim_fanc, ag_am_tim_fplus = butterfly_quantification(
    abba_num=ag_am_tim_abba,
    baba_num=ag_am_tim_baba,
    baaa_num=ag_am_tim_baaa,
    abaa_num=ag_am_tim_abaa,
    abba_den=ag_tim_tim_abba,
    baba_den=ag_tim_tim_baba,
    baaa_den=ag_tim_tim_baaa,
    abaa_den=ag_tim_tim_abaa,
)
# Load in the bootstrapped distributions.
ag_am_tim_bs_dict = load_bs_stds(
    p1='ag',
    p2='am',
    p3='timP',
)
# Calculate p-values for introgression detection method.
ag_am_tim_pval_dict = calc_p_values(
    obs_d=ag_am_tim_d,
    obs_danc=ag_am_tim_danc,
    obs_dplus=ag_am_tim_dplus,
    bs_std_d=ag_am_tim_bs_dict['d'],
    bs_std_danc=ag_am_tim_bs_dict['danc'],
    bs_std_dplus=ag_am_tim_bs_dict['dplus'],
)
# Print a summary for this trio.
print('Summary for time period 4 ((H. m. aglaope, H. m. amaryllis), H. timareta)')
print('*** Site Pattern Counts ***')
print('ABBA - BABA: {0}'.format(round(ag_am_tim_abba_baba, 3)))
print('BAAA - ABAA: {0}'.format(round(ag_am_tim_baaa_abaa, 3)))
print('*** Detection Metrics ***')
print('D - p-value: {0} - {1}'.format(round(ag_am_tim_d, 3), round(ag_am_tim_pval_dict['d'], 3)))
print('Danc - p-value: {0} - {1}'.format(round(ag_am_tim_danc, 3), round(ag_am_tim_pval_dict['danc'], 3)))
print('D+ - p-value: {0} - {1}'.format(round(ag_am_tim_dplus, 3), round(ag_am_tim_pval_dict['dplus'], 3)))
print('*** Quantification Metrics ***')
print('fhom - std: {0} - {1}'.format(round(ag_am_tim_fhom, 3), round(ag_am_tim_bs_dict['fhom'], 3)))
print('fanc - std: {0} - {1}'.format(round(ag_am_tim_fanc, 3), round(ag_am_tim_bs_dict['fanc'], 3)))
print('f+ - std: {0} - {1}'.format(round(ag_am_tim_fplus, 3), round(ag_am_tim_bs_dict['fplus'], 3)))

Summary for time period 4 ((H. m. aglaope, H. m. amaryllis), H. timareta)
*** Site Pattern Counts ***
ABBA - BABA: 2413.309
BAAA - ABAA: 959.368
*** Detection Metrics ***
D - p-value: 0.039 - 0.0
Danc - p-value: 0.003 - 0.008
D+ - p-value: 0.008 - 0.0
*** Quantification Metrics ***
fhom - std: 0.018 - 0.003
fanc - std: 0.008 - 0.003
f+ - std: 0.013 - 0.003


Next, we will look at the intermediate time period 4 + 3—i.e., ((_H. m. melpomene (French Guianna)_, _H. m. amaryllis_), _H. timareta_).

In [31]:
# Calculate site pattern counts for.
melp_fg_am_tim_abba, melp_fg_am_tim_baba, melp_fg_am_tim_baaa, melp_fg_am_tim_abaa = butterfly_site_patterns(
    p1=melp_fg_der_freq_array,
    p2=amaryllis_der_freq_array,
    p3=timareta_der_freq_array,
)
melp_fg_tim_tim_abba, melp_fg_tim_tim_baba, melp_fg_tim_tim_baaa, melp_fg_tim_tim_abaa = butterfly_site_patterns(
    p1=melp_fg_der_freq_array,
    p2=timareta_der_freq_array,
    p3=timareta_der_freq_array,
)
# Calculate site pattern differences.
melp_fg_am_tim_abba_baba = melp_fg_am_tim_abba - melp_fg_am_tim_baba
melp_fg_am_tim_baaa_abaa = melp_fg_am_tim_baaa - melp_fg_am_tim_abaa
# Calculate introgression detection metrics.
melp_fg_am_tim_d, melp_fg_am_tim_danc, melp_fg_am_tim_dplus = butterfly_detection(
    abba=melp_fg_am_tim_abba,
    baba=melp_fg_am_tim_baba,
    baaa=melp_fg_am_tim_baaa,
    abaa=melp_fg_am_tim_abaa,
)
# Calculate introgression quantification metrics.
melp_fg_am_tim_fhom, melp_fg_am_tim_fanc, melp_fg_am_tim_fplus = butterfly_quantification(
    abba_num=melp_fg_am_tim_abba,
    baba_num=melp_fg_am_tim_baba,
    baaa_num=melp_fg_am_tim_baaa,
    abaa_num=melp_fg_am_tim_abaa,
    abba_den=ag_tim_tim_abba,
    baba_den=ag_tim_tim_baba,
    baaa_den=ag_tim_tim_baaa,
    abaa_den=ag_tim_tim_abaa,
)
# Load in the bootstrapped distributions.
melp_fg_am_tim_bs_dict = load_bs_stds(
    p1='melFG',
    p2='am',
    p3='timP',
)
# Calculate p-values for introgression detection method.
melp_fg_am_tim_pval_dict = calc_p_values(
    obs_d=melp_fg_am_tim_d,
    obs_danc=melp_fg_am_tim_danc,
    obs_dplus=melp_fg_am_tim_dplus,
    bs_std_d=melp_fg_am_tim_bs_dict['d'],
    bs_std_danc=melp_fg_am_tim_bs_dict['danc'],
    bs_std_dplus=melp_fg_am_tim_bs_dict['dplus'],
)
# Print a summary for this trio.
print('Summary for time period 4 + 3 ((H. m. melpomene (French Guianna), H. m. amaryllis), H. timareta)')
print('*** Site Pattern Counts ***')
print('ABBA - BABA: {0}'.format(round(melp_fg_am_tim_abba_baba, 3)))
print('BAAA - ABAA: {0}'.format(round(melp_fg_am_tim_baaa_abaa, 3)))
print('*** Detection Metrics ***')
print('D - p-value: {0} - {1}'.format(round(melp_fg_am_tim_d, 3), round(melp_fg_am_tim_pval_dict['d'], 3)))
print('Danc - p-value: {0} - {1}'.format(round(melp_fg_am_tim_danc, 3), round(melp_fg_am_tim_pval_dict['danc'], 3)))
print('D+ - p-value: {0} - {1}'.format(round(melp_fg_am_tim_dplus, 3), round(melp_fg_am_tim_pval_dict['dplus'], 3)))
print('*** Quantification Metrics ***')
print('fhom - std: {0} - {1}'.format(round(melp_fg_am_tim_fhom, 3), round(melp_fg_am_tim_bs_dict['fhom'], 3)))
print('fanc - std: {0} - {1}'.format(round(melp_fg_am_tim_fanc, 3), round(melp_fg_am_tim_bs_dict['fanc'], 3)))
print('f+ - std: {0} - {1}'.format(round(melp_fg_am_tim_fplus, 3), round(melp_fg_am_tim_bs_dict['fplus'], 3)))

Summary for time period 4 + 3 ((H. m. melpomene (French Guianna), H. m. amaryllis), H. timareta)
*** Site Pattern Counts ***
ABBA - BABA: 14143.45
BAAA - ABAA: 25927.758
*** Detection Metrics ***
D - p-value: 0.197 - 0.0
Danc - p-value: 0.061 - 0.0
D+ - p-value: 0.081 - 0.0
*** Quantification Metrics ***
fhom - std: 0.105 - 0.005
fanc - std: 0.22 - 0.008
f+ - std: 0.158 - 0.006


Next, we will look at the long time period 4 + 3 + 2—i.e., ((_H. m. melpomene (Panama)_, _H. m. amaryllis_), _H. timareta_).

In [32]:
# Calculate site pattern counts for.
melp_p_am_tim_abba, melp_p_am_tim_baba, melp_p_am_tim_baaa, melp_p_am_tim_abaa = butterfly_site_patterns(
    p1=melp_p_der_freq_array,
    p2=amaryllis_der_freq_array,
    p3=timareta_der_freq_array,
)
melp_p_tim_tim_abba, melp_p_tim_tim_baba, melp_p_tim_tim_baaa, melp_p_tim_tim_abaa = butterfly_site_patterns(
    p1=melp_p_der_freq_array,
    p2=timareta_der_freq_array,
    p3=timareta_der_freq_array,
)
# Calculate site pattern differences.
melp_p_am_tim_abba_baba = melp_p_am_tim_abba - melp_p_am_tim_baba
melp_p_am_tim_baaa_abaa = melp_p_am_tim_baaa - melp_p_am_tim_abaa
# Calculate introgression detection metrics.
melp_p_am_tim_d, melp_p_am_tim_danc, melp_p_am_tim_dplus = butterfly_detection(
    abba=melp_p_am_tim_abba,
    baba=melp_p_am_tim_baba,
    baaa=melp_p_am_tim_baaa,
    abaa=melp_p_am_tim_abaa,
)
# Calculate introgression quantification metrics.
melp_p_am_tim_fhom, melp_p_am_tim_fanc, melp_p_am_tim_fplus = butterfly_quantification(
    abba_num=melp_p_am_tim_abba,
    baba_num=melp_p_am_tim_baba,
    baaa_num=melp_p_am_tim_baaa,
    abaa_num=melp_p_am_tim_abaa,
    abba_den=ag_tim_tim_abba,
    baba_den=ag_tim_tim_baba,
    baaa_den=ag_tim_tim_baaa,
    abaa_den=ag_tim_tim_abaa,
)
# Load in the bootstrapped distributions.
melp_p_am_tim_bs_dict = load_bs_stds(
    p1='melP',
    p2='am',
    p3='timP',
)
# Calculate p-values for introgression detection method.
melp_p_am_tim_pval_dict = calc_p_values(
    obs_d=melp_p_am_tim_d,
    obs_danc=melp_p_am_tim_danc,
    obs_dplus=melp_p_am_tim_dplus,
    bs_std_d=melp_p_am_tim_bs_dict['d'],
    bs_std_danc=melp_p_am_tim_bs_dict['danc'],
    bs_std_dplus=melp_p_am_tim_bs_dict['dplus'],
)
# Print a summary for this trio.
print('Summary for time period 4 + 3 + 2 ((H. m. melpomene (Panama), H. m. amaryllis), H. timareta)')
print('*** Site Pattern Counts ***')
print('ABBA - BABA: {0}'.format(round(melp_p_am_tim_abba_baba, 3)))
print('BAAA - ABAA: {0}'.format(round(melp_p_am_tim_baaa_abaa, 3)))
print('*** Detection Metrics ***')
print('D - p-value: {0} - {1}'.format(round(melp_p_am_tim_d, 3), round(melp_p_am_tim_pval_dict['d'], 3)))
print('Danc - p-value: {0} - {1}'.format(round(melp_p_am_tim_danc, 3), round(melp_p_am_tim_pval_dict['danc'], 3)))
print('D+ - p-value: {0} - {1}'.format(round(melp_p_am_tim_dplus, 3), round(melp_p_am_tim_pval_dict['dplus'], 3)))
print('*** Quantification Metrics ***')
print('fhom - std: {0} - {1}'.format(round(melp_p_am_tim_fhom, 3), round(melp_p_am_tim_bs_dict['fhom'], 3)))
print('fanc - std: {0} - {1}'.format(round(melp_p_am_tim_fanc, 3), round(melp_p_am_tim_bs_dict['fanc'], 3)))
print('f+ - std: {0} - {1}'.format(round(melp_p_am_tim_fplus, 3), round(melp_p_am_tim_bs_dict['fplus'], 3)))

Summary for time period 4 + 3 + 2 ((H. m. melpomene (Panama), H. m. amaryllis), H. timareta)
*** Site Pattern Counts ***
ABBA - BABA: 20634.37
BAAA - ABAA: -21659.164
*** Detection Metrics ***
D - p-value: 0.229 - 0.0
Danc - p-value: -0.051 - 0.0
D+ - p-value: -0.002 - 0.334
*** Quantification Metrics ***
fhom - std: 0.153 - 0.01
fanc - std: -0.184 - 0.013
f+ - std: -0.004 - 0.009


Next, we will look at the long time period 4 + 3 + 2—i.e., ((_H. m. rosina_, _H. m. amaryllis_), _H. timareta_).

In [33]:
# Calculate site pattern counts for.
rosina_am_tim_abba, rosina_am_tim_baba, rosina_am_tim_baaa, rosina_am_tim_abaa = butterfly_site_patterns(
    p1=rosina_der_freq_array,
    p2=amaryllis_der_freq_array,
    p3=timareta_der_freq_array,
)
rosina_tim_tim_abba, rosina_tim_tim_baba, rosina_tim_tim_baaa, rosina_tim_tim_abaa = butterfly_site_patterns(
    p1=rosina_der_freq_array,
    p2=timareta_der_freq_array,
    p3=timareta_der_freq_array,
)
# Calculate site pattern differences.
rosina_am_tim_abba_baba = rosina_am_tim_abba - rosina_am_tim_baba
rosina_am_tim_baaa_abaa = rosina_am_tim_baaa - rosina_am_tim_abaa
# Calculate introgression detection metrics.
rosina_am_tim_d, rosina_am_tim_danc, rosina_am_tim_dplus = butterfly_detection(
    abba=rosina_am_tim_abba,
    baba=rosina_am_tim_baba,
    baaa=rosina_am_tim_baaa,
    abaa=rosina_am_tim_abaa,
)
# Calculate introgression quantification metrics.
rosina_am_tim_fhom, rosina_am_tim_fanc, rosina_am_tim_fplus = butterfly_quantification(
    abba_num=rosina_am_tim_abba,
    baba_num=rosina_am_tim_baba,
    baaa_num=rosina_am_tim_baaa,
    abaa_num=rosina_am_tim_abaa,
    abba_den=ag_tim_tim_abba,
    baba_den=ag_tim_tim_baba,
    baaa_den=ag_tim_tim_baaa,
    abaa_den=ag_tim_tim_abaa,
)
# Load in the bootstrapped distributions.
rosina_am_tim_bs_dict = load_bs_stds(
    p1='ros',
    p2='am',
    p3='timP',
)
# Calculate p-values for introgression detection method.
rosina_am_tim_pval_dict = calc_p_values(
    obs_d=rosina_am_tim_d,
    obs_danc=rosina_am_tim_danc,
    obs_dplus=rosina_am_tim_dplus,
    bs_std_d=rosina_am_tim_bs_dict['d'],
    bs_std_danc=rosina_am_tim_bs_dict['danc'],
    bs_std_dplus=rosina_am_tim_bs_dict['dplus'],
)
# Print a summary for this trio.
print('Summary for time period 4 + 3 + 2 ((H. m. rosina, H. m. amaryllis), H. timareta)')
print('*** Site Pattern Counts ***')
print('ABBA - BABA: {0}'.format(round(rosina_am_tim_abba_baba, 3)))
print('BAAA - ABAA: {0}'.format(round(rosina_am_tim_baaa_abaa, 3)))
print('*** Detection Metrics ***')
print('D - p-value: {0} - {1}'.format(round(rosina_am_tim_d, 3), round(rosina_am_tim_pval_dict['d'], 3)))
print('Danc - p-value: {0} - {1}'.format(round(rosina_am_tim_danc, 3), round(rosina_am_tim_pval_dict['danc'], 3)))
print('D+ - p-value: {0} - {1}'.format(round(rosina_am_tim_dplus, 3), round(rosina_am_tim_pval_dict['dplus'], 3)))
print('*** Quantification Metrics ***')
print('fhom - std: {0} - {1}'.format(round(rosina_am_tim_fhom, 3), round(rosina_am_tim_bs_dict['fhom'], 3)))
print('fanc - std: {0} - {1}'.format(round(rosina_am_tim_fanc, 3), round(rosina_am_tim_bs_dict['fanc'], 3)))
print('f+ - std: {0} - {1}'.format(round(rosina_am_tim_fplus, 3), round(rosina_am_tim_bs_dict['fplus'], 3)))

Summary for time period 4 + 3 + 2 ((H. m. rosina, H. m. amaryllis), H. timareta)
*** Site Pattern Counts ***
ABBA - BABA: 19153.575
BAAA - ABAA: -1221.242
*** Detection Metrics ***
D - p-value: 0.209 - 0.0
Danc - p-value: -0.003 - 0.2
D+ - p-value: 0.033 - 0.0
*** Quantification Metrics ***
fhom - std: 0.142 - 0.01
fanc - std: -0.01 - 0.012
f+ - std: 0.071 - 0.01


## Introgression Between  _H. m. rosina_ & _H. cydno_
Next, we re-examined rates of gene flow between _H. m. rosina_ and _H. cydno_ across two time periods: a short, recent period, and a long period.
  
First, we will look at the short time period 4—i.e., ((_H. m. melpomene (Panama)_, _H. m. rosina_), _H. cydno_).

In [34]:
# Calculate site pattern counts for.
melp_p_rosina_cydno_abba, melp_p_rosina_cydno_baba, melp_p_rosina_cydno_baaa, melp_p_rosina_cydno_abaa = butterfly_site_patterns(
    p1=melp_p_der_freq_array,
    p2=rosina_der_freq_array,
    p3=cydno_der_freq_array,
)
melp_p_cydno_cydno_abba, melp_p_cydno_cydno_baba, melp_p_cydno_cydno_baaa, melp_p_cydno_cydno_abaa = butterfly_site_patterns(
    p1=melp_p_der_freq_array,
    p2=cydno_der_freq_array,
    p3=cydno_der_freq_array,
)
# Calculate site pattern differences.
melp_p_rosina_cydno_abba_baba = melp_p_rosina_cydno_abba - melp_p_rosina_cydno_baba
melp_p_rosina_cydno_baaa_abaa = melp_p_rosina_cydno_baaa - melp_p_rosina_cydno_abaa
# Calculate introgression detection metrics.
melp_p_rosina_cydno_d, melp_p_rosina_cydno_danc, melp_p_rosina_cydno_dplus = butterfly_detection(
    abba=melp_p_rosina_cydno_abba,
    baba=melp_p_rosina_cydno_baba,
    baaa=melp_p_rosina_cydno_baaa,
    abaa=melp_p_rosina_cydno_abaa,
)
# Calculate introgression quantification metrics.
melp_p_rosina_cydno_fhom, melp_p_rosina_cydno_fanc, melp_p_rosina_cydno_fplus = butterfly_quantification(
    abba_num=melp_p_rosina_cydno_abba,
    baba_num=melp_p_rosina_cydno_baba,
    baaa_num=melp_p_rosina_cydno_baaa,
    abaa_num=melp_p_rosina_cydno_abaa,
    abba_den=ag_tim_tim_abba,
    baba_den=ag_tim_tim_baba,
    baaa_den=ag_tim_tim_baaa,
    abaa_den=ag_tim_tim_abaa,
)
# Load in the bootstrapped distributions.
melp_p_rosina_cydno_bs_dict = load_bs_stds(
    p1='melP',
    p2='ros',
    p3='chi',
)
# Calculate p-values for introgression detection method.
melp_p_rosina_cydno_pval_dict = calc_p_values(
    obs_d=melp_p_rosina_cydno_d,
    obs_danc=melp_p_rosina_cydno_danc,
    obs_dplus=melp_p_rosina_cydno_dplus,
    bs_std_d=melp_p_rosina_cydno_bs_dict['d'],
    bs_std_danc=melp_p_rosina_cydno_bs_dict['danc'],
    bs_std_dplus=melp_p_rosina_cydno_bs_dict['dplus'],
)
# Print a summary for this trio.
print('Summary for time period 4 ((H. m. melpomene (Panama), H. m. rosina), H. cydno)')
print('*** Site Pattern Counts ***')
print('ABBA - BABA: {0}'.format(round(melp_p_rosina_cydno_abba_baba, 3)))
print('BAAA - ABAA: {0}'.format(round(melp_p_rosina_cydno_baaa_abaa, 3)))
print('*** Detection Metrics ***')
print('D - p-value: {0} - {1}'.format(round(melp_p_rosina_cydno_d, 3), round(melp_p_rosina_cydno_pval_dict['d'], 3)))
print('Danc - p-value: {0} - {1}'.format(round(melp_p_rosina_cydno_danc, 3), round(melp_p_rosina_cydno_pval_dict['danc'], 3)))
print('D+ - p-value: {0} - {1}'.format(round(melp_p_rosina_cydno_dplus, 3), round(melp_p_rosina_cydno_pval_dict['dplus'], 3)))
print('*** Quantification Metrics ***')
print('fhom - std: {0} - {1}'.format(round(melp_p_rosina_cydno_fhom, 3), round(melp_p_rosina_cydno_bs_dict['fhom'], 3)))
print('fanc - std: {0} - {1}'.format(round(melp_p_rosina_cydno_fanc, 3), round(melp_p_rosina_cydno_bs_dict['fanc'], 3)))
print('f+ - std: {0} - {1}'.format(round(melp_p_rosina_cydno_fplus, 3), round(melp_p_rosina_cydno_bs_dict['fplus'], 3)))

Summary for time period 4 ((H. m. melpomene (Panama), H. m. rosina), H. cydno)
*** Site Pattern Counts ***
ABBA - BABA: 4194.587
BAAA - ABAA: -17724.129
*** Detection Metrics ***
D - p-value: 0.073 - 0.0
Danc - p-value: -0.058 - 0.0
D+ - p-value: -0.037 - 0.0
*** Quantification Metrics ***
fhom - std: 0.031 - 0.002
fanc - std: -0.15 - 0.037
f+ - std: -0.054 - 0.005


Next, we will look at the long time period 4 + 3 + 2—i.e., ((_H. m. melpomene (French Guiana)_, _H. m. rosina_), _H. cydno_).

In [35]:
# Calculate site pattern counts for.
melp_fg_rosina_cydno_abba, melp_fg_rosina_cydno_baba, melp_fg_rosina_cydno_baaa, melp_fg_rosina_cydno_abaa = butterfly_site_patterns(
    p1=melp_fg_der_freq_array,
    p2=rosina_der_freq_array,
    p3=cydno_der_freq_array,
)
melp_fg_cydno_cydno_abba, melp_fg_cydno_cydno_baba, melp_fg_cydno_cydno_baaa, melp_fg_cydno_cydno_abaa = butterfly_site_patterns(
    p1=melp_fg_der_freq_array,
    p2=cydno_der_freq_array,
    p3=cydno_der_freq_array,
)
# Calculate site pattern differences.
melp_fg_rosina_cydno_abba_baba = melp_fg_rosina_cydno_abba - melp_fg_rosina_cydno_baba
melp_fg_rosina_cydno_baaa_abaa = melp_fg_rosina_cydno_baaa - melp_fg_rosina_cydno_abaa
# Calculate introgression detection metrics.
melp_fg_rosina_cydno_d, melp_fg_rosina_cydno_danc, melp_fg_rosina_cydno_dplus = butterfly_detection(
    abba=melp_fg_rosina_cydno_abba,
    baba=melp_fg_rosina_cydno_baba,
    baaa=melp_fg_rosina_cydno_baaa,
    abaa=melp_fg_rosina_cydno_abaa,
)
# Calculate introgression quantification metrics.
melp_fg_rosina_cydno_fhom, melp_fg_rosina_cydno_fanc, melp_fg_rosina_cydno_fplus = butterfly_quantification(
    abba_num=melp_fg_rosina_cydno_abba,
    baba_num=melp_fg_rosina_cydno_baba,
    baaa_num=melp_fg_rosina_cydno_baaa,
    abaa_num=melp_fg_rosina_cydno_abaa,
    abba_den=ag_tim_tim_abba,
    baba_den=ag_tim_tim_baba,
    baaa_den=ag_tim_tim_baaa,
    abaa_den=ag_tim_tim_abaa,
)
# Load in the bootstrapped distributions.
melp_fg_rosina_cydno_bs_dict = load_bs_stds(
    p1='melFG',
    p2='ros',
    p3='chi',
)
# Calculate p-values for introgression detection method.
melp_fg_rosina_cydno_pval_dict = calc_p_values(
    obs_d=melp_fg_rosina_cydno_d,
    obs_danc=melp_fg_rosina_cydno_danc,
    obs_dplus=melp_fg_rosina_cydno_dplus,
    bs_std_d=melp_fg_rosina_cydno_bs_dict['d'],
    bs_std_danc=melp_fg_rosina_cydno_bs_dict['danc'],
    bs_std_dplus=melp_fg_rosina_cydno_bs_dict['dplus'],
)
# Print a summary for this trio.
print('Summary for time period 4 + 3 + 2 ((H. m. melpomene (French Guiana), H. m. rosina), H. cydno)')
print('*** Site Pattern Counts ***')
print('ABBA - BABA: {0}'.format(round(melp_fg_rosina_cydno_abba_baba, 3)))
print('BAAA - ABAA: {0}'.format(round(melp_fg_rosina_cydno_baaa_abaa, 3)))
print('*** Detection Metrics ***')
print('D - p-value: {0} - {1}'.format(round(melp_fg_rosina_cydno_d, 3), round(melp_fg_rosina_cydno_pval_dict['d'], 3)))
print('Danc - p-value: {0} - {1}'.format(round(melp_fg_rosina_cydno_danc, 3), round(melp_fg_rosina_cydno_pval_dict['danc'], 3)))
print('D+ - p-value: {0} - {1}'.format(round(melp_fg_rosina_cydno_dplus, 3), round(melp_fg_rosina_cydno_pval_dict['dplus'], 3)))
print('*** Quantification Metrics ***')
print('fhom - std: {0} - {1}'.format(round(melp_fg_rosina_cydno_fhom, 3), round(melp_fg_rosina_cydno_bs_dict['fhom'], 3)))
print('fanc - std: {0} - {1}'.format(round(melp_fg_rosina_cydno_fanc, 3), round(melp_fg_rosina_cydno_bs_dict['fanc'], 3)))
print('f+ - std: {0} - {1}'.format(round(melp_fg_rosina_cydno_fplus, 3), round(melp_fg_rosina_cydno_bs_dict['fplus'], 3)))

Summary for time period 4 + 3 + 2 ((H. m. melpomene (French Guiana), H. m. rosina), H. cydno)
*** Site Pattern Counts ***
ABBA - BABA: 34801.297
BAAA - ABAA: 66960.422
*** Detection Metrics ***
D - p-value: 0.493 - 0.0
Danc - p-value: 0.147 - 0.0
D+ - p-value: 0.194 - 0.0
*** Quantification Metrics ***
fhom - std: 0.258 - 0.007
fanc - std: 0.568 - 0.008
f+ - std: 0.403 - 0.006


Next, we will look at the long time period 4 + 3 + 2—i.e., ((_H. m. amaryllis_, _H. m. rosina_), _H. cydno_).

In [36]:
# Calculate site pattern counts for.
amaryllis_rosina_cydno_abba, amaryllis_rosina_cydno_baba, amaryllis_rosina_cydno_baaa, amaryllis_rosina_cydno_abaa = butterfly_site_patterns(
    p1=amaryllis_der_freq_array,
    p2=rosina_der_freq_array,
    p3=cydno_der_freq_array,
)
amaryllis_cydno_cydno_abba, amaryllis_cydno_cydno_baba, amaryllis_cydno_cydno_baaa, amaryllis_cydno_cydno_abaa = butterfly_site_patterns(
    p1=amaryllis_der_freq_array,
    p2=cydno_der_freq_array,
    p3=cydno_der_freq_array,
)
# Calculate site pattern differences.
amaryllis_rosina_cydno_abba_baba = amaryllis_rosina_cydno_abba - amaryllis_rosina_cydno_baba
amaryllis_rosina_cydno_baaa_abaa = amaryllis_rosina_cydno_baaa - amaryllis_rosina_cydno_abaa
# Calculate introgression detection metrics.
amaryllis_rosina_cydno_d, amaryllis_rosina_cydno_danc, amaryllis_rosina_cydno_dplus = butterfly_detection(
    abba=amaryllis_rosina_cydno_abba,
    baba=amaryllis_rosina_cydno_baba,
    baaa=amaryllis_rosina_cydno_baaa,
    abaa=amaryllis_rosina_cydno_abaa,
)
# Calculate introgression quantification metrics.
amaryllis_rosina_cydno_fhom, amaryllis_rosina_cydno_fanc, amaryllis_rosina_cydno_fplus = butterfly_quantification(
    abba_num=amaryllis_rosina_cydno_abba,
    baba_num=amaryllis_rosina_cydno_baba,
    baaa_num=amaryllis_rosina_cydno_baaa,
    abaa_num=amaryllis_rosina_cydno_abaa,
    abba_den=ag_tim_tim_abba,
    baba_den=ag_tim_tim_baba,
    baaa_den=ag_tim_tim_baaa,
    abaa_den=ag_tim_tim_abaa,
)
# Load in the bootstrapped distributions.
amaryllis_rosina_cydno_bs_dict = load_bs_stds(
    p1='am',
    p2='ros',
    p3='chi',
)
# Calculate p-values for introgression detection method.
amaryllis_rosina_cydno_pval_dict = calc_p_values(
    obs_d=amaryllis_rosina_cydno_d,
    obs_danc=amaryllis_rosina_cydno_danc,
    obs_dplus=amaryllis_rosina_cydno_dplus,
    bs_std_d=amaryllis_rosina_cydno_bs_dict['d'],
    bs_std_danc=amaryllis_rosina_cydno_bs_dict['danc'],
    bs_std_dplus=amaryllis_rosina_cydno_bs_dict['dplus'],
)
# Print a summary for this trio.
print('Summary for time period 4 + 3 + 2 ((H. m. amaryllis, H. m. rosina), H. cydno)')
print('*** Site Pattern Counts ***')
print('ABBA - BABA: {0}'.format(round(amaryllis_rosina_cydno_abba_baba, 3)))
print('BAAA - ABAA: {0}'.format(round(amaryllis_rosina_cydno_baaa_abaa, 3)))
print('*** Detection Metrics ***')
print('D - p-value: {0} - {1}'.format(round(amaryllis_rosina_cydno_d, 3), round(amaryllis_rosina_cydno_pval_dict['d'], 3)))
print('Danc - p-value: {0} - {1}'.format(round(amaryllis_rosina_cydno_danc, 3), round(amaryllis_rosina_cydno_pval_dict['danc'], 3)))
print('D+ - p-value: {0} - {1}'.format(round(amaryllis_rosina_cydno_dplus, 3), round(amaryllis_rosina_cydno_pval_dict['dplus'], 3)))
print('*** Quantification Metrics ***')
print('fhom - std: {0} - {1}'.format(round(amaryllis_rosina_cydno_fhom, 3), round(amaryllis_rosina_cydno_bs_dict['fhom'], 3)))
print('fanc - std: {0} - {1}'.format(round(amaryllis_rosina_cydno_fanc, 3), round(amaryllis_rosina_cydno_bs_dict['fanc'], 3)))
print('f+ - std: {0} - {1}'.format(round(amaryllis_rosina_cydno_fplus, 3), round(amaryllis_rosina_cydno_bs_dict['fplus'], 3)))

Summary for time period 4 + 3 + 2 ((H. m. amaryllis, H. m. rosina), H. cydno)
*** Site Pattern Counts ***
ABBA - BABA: 37885.734
BAAA - ABAA: 58260.551
*** Detection Metrics ***
D - p-value: 0.49 - 0.0
Danc - p-value: 0.127 - 0.0
D+ - p-value: 0.179 - 0.0
*** Quantification Metrics ***
fhom - std: 0.281 - 0.007
fanc - std: 0.494 - 0.008
f+ - std: 0.38 - 0.007


Lastly, we will look at the long time period 4 + 3 + 2—i.e., ((_H. m. aglaope_, _H. m. rosina_), _H. cydno_).

In [37]:
# Calculate site pattern counts for.
aglaope_rosina_cydno_abba, aglaope_rosina_cydno_baba, aglaope_rosina_cydno_baaa, aglaope_rosina_cydno_abaa = butterfly_site_patterns(
    p1=aglaope_der_freq_array,
    p2=rosina_der_freq_array,
    p3=cydno_der_freq_array,
)
aglaope_cydno_cydno_abba, aglaope_cydno_cydno_baba, aglaope_cydno_cydno_baaa, aglaope_cydno_cydno_abaa = butterfly_site_patterns(
    p1=aglaope_der_freq_array,
    p2=cydno_der_freq_array,
    p3=cydno_der_freq_array,
)
# Calculate site pattern differences.
aglaope_rosina_cydno_abba_baba = aglaope_rosina_cydno_abba - aglaope_rosina_cydno_baba
aglaope_rosina_cydno_baaa_abaa = aglaope_rosina_cydno_baaa - aglaope_rosina_cydno_abaa
# Calculate introgression detection metrics.
aglaope_rosina_cydno_d, aglaope_rosina_cydno_danc, aglaope_rosina_cydno_dplus = butterfly_detection(
    abba=aglaope_rosina_cydno_abba,
    baba=aglaope_rosina_cydno_baba,
    baaa=aglaope_rosina_cydno_baaa,
    abaa=aglaope_rosina_cydno_abaa,
)
# Calculate introgression quantification metrics.
aglaope_rosina_cydno_fhom, aglaope_rosina_cydno_fanc, aglaope_rosina_cydno_fplus = butterfly_quantification(
    abba_num=aglaope_rosina_cydno_abba,
    baba_num=aglaope_rosina_cydno_baba,
    baaa_num=aglaope_rosina_cydno_baaa,
    abaa_num=aglaope_rosina_cydno_abaa,
    abba_den=ag_tim_tim_abba,
    baba_den=ag_tim_tim_baba,
    baaa_den=ag_tim_tim_baaa,
    abaa_den=ag_tim_tim_abaa,
)
# Load in the bootstrapped distributions.
aglaope_rosina_cydno_bs_dict = load_bs_stds(
    p1='ag',
    p2='ros',
    p3='chi',
)
# Calculate p-values for introgression detection method.
aglaope_rosina_cydno_pval_dict = calc_p_values(
    obs_d=aglaope_rosina_cydno_d,
    obs_danc=aglaope_rosina_cydno_danc,
    obs_dplus=aglaope_rosina_cydno_dplus,
    bs_std_d=aglaope_rosina_cydno_bs_dict['d'],
    bs_std_danc=aglaope_rosina_cydno_bs_dict['danc'],
    bs_std_dplus=aglaope_rosina_cydno_bs_dict['dplus'],
)
# Print a summary for this trio.
print('Summary for time period 4 + 3 + 2 ((H. m. aglaope, H. m. rosina), H. cydno)')
print('*** Site Pattern Counts ***')
print('ABBA - BABA: {0}'.format(round(aglaope_rosina_cydno_abba_baba, 3)))
print('BAAA - ABAA: {0}'.format(round(aglaope_rosina_cydno_baaa_abaa, 3)))
print('*** Detection Metrics ***')
print('D - p-value: {0} - {1}'.format(round(aglaope_rosina_cydno_d, 3), round(aglaope_rosina_cydno_pval_dict['d'], 3)))
print('Danc - p-value: {0} - {1}'.format(round(aglaope_rosina_cydno_danc, 3), round(aglaope_rosina_cydno_pval_dict['danc'], 3)))
print('D+ - p-value: {0} - {1}'.format(round(aglaope_rosina_cydno_dplus, 3), round(aglaope_rosina_cydno_pval_dict['dplus'], 3)))
print('*** Quantification Metrics ***')
print('fhom - std: {0} - {1}'.format(round(aglaope_rosina_cydno_fhom, 3), round(aglaope_rosina_cydno_bs_dict['fhom'], 3)))
print('fanc - std: {0} - {1}'.format(round(aglaope_rosina_cydno_fanc, 3), round(aglaope_rosina_cydno_bs_dict['fanc'], 3)))
print('f+ - std: {0} - {1}'.format(round(aglaope_rosina_cydno_fplus, 3), round(aglaope_rosina_cydno_bs_dict['fplus'], 3)))

Summary for time period 4 + 3 + 2 ((H. m. aglaope, H. m. rosina), H. cydno)
*** Site Pattern Counts ***
ABBA - BABA: 38524.203
BAAA - ABAA: 57445.078
*** Detection Metrics ***
D - p-value: 0.501 - 0.0
Danc - p-value: 0.126 - 0.0
D+ - p-value: 0.18 - 0.0
*** Quantification Metrics ***
fhom - std: 0.286 - 0.007
fanc - std: 0.487 - 0.009
f+ - std: 0.38 - 0.007


## Bootstrapping example

In [42]:
import random
import time
def butterfly_bootstrap(
    genome_df,
    p1_id,
    p2_id,
    p3_id,
    n_replicates,
):
    """
    ###########################################################################
    INPUT
        genome_df: Whole-genome dataframe for butterflies.
        p1_id: ID for the P1 population.
        p2_id: ID for the P2 population.
        p3_id: ID for the P3 population.
        n_replicates: Number of bootstrap replicates.
    ---------------------------------------------------------------------------
    OUTPUT: List of site pattern counts from bootstrapped replicates.
    ###########################################################################
    """
    # Intialize arrays to store bootstrapped values.
    abba_array = np.array([])
    baba_array = np.array([])
    baaa_array = np.array([])
    abaa_array = np.array([])
    abba_hom_array = np.array([])
    baba_hom_array = np.array([])
    baaa_hom_array = np.array([])
    abaa_hom_array = np.array([])
    # Intialize a dictionary of chromosome lengths from table S4.6.1 (doi:10.1038/nature11041).
    chromosome_dicc = {
        'chr1': 15_755_848, 'chr2': 3_590_614, 'chr3': 8_943_778,
        'chr4': 6_670_749, 'chr5': 8_000_463, 'chr6': 13_155_959,
        'chr7': 11_792_147, 'chr8': 6_828_437, 'chr9': 8_311_378,
        'chr10': 17_492_759, 'chr11': 11_233_900, 'chr12': 15_835_627,
        'chr13': 13_526_828, 'chr14': 6_527_076, 'chr15': 7_742_867,
        'chr16': 9_269_283, 'chr17': 13_935_249, 'chr18': 15_453_272,
        'chr19': 14_778_520, 'chr20': 5_748_428, 'chrZ': 4_088_377,
    }
    # For each bootstrap replicate.
    for i in range(n_replicates):
        # Intialize empty arrays to store bootstrapped genomes.
        bootstrapped_p1 = np.array([])
        bootstrapped_p2 = np.array([])
        bootstrapped_p3 = np.array([])
        # For each bootstrap window (219 x 1 Mb).
        for j in range(219):
            # Randomly select a chromosome.
            chromosome = random.choice(list(chromosome_dicc.keys()))
            # Randomly generate a start and end position.
            start = np.random.randint((chromosome_dicc[chromosome] - 999_999))
            end = start + 1_000_000
            # Subset the whole-genome dataframe to only include the randomly selected chromosome.
            chromosome_df = genome_df[genome_df['chromosome'] == chromosome]
            # Extract the variable positions for that chromosome.
            variable_positions = chromosome_df['chrom_pos'].values
            # Identify the variants that fall within the 1 Mb window.
            variants = np.where(((start <= variable_positions) & (variable_positions <= end)))[0]
            # Extract the focal populations to subset.
            p1_chromosome = chromosome_df[p1_id].values
            p2_chromosome = chromosome_df[p2_id].values
            p3_chromosome = chromosome_df[p3_id].values
            # Subset the 1 Mb window from each focal population.
            p1_subset = p1_chromosome[variants]
            p2_subset = p2_chromosome[variants]
            p3_subset = p3_chromosome[variants]
            # Append 219 windows of 1 Mb to make bootstrapped genomes.
            bootstrapped_p1 = np.append(bootstrapped_p1, p1_subset)
            bootstrapped_p2 = np.append(bootstrapped_p2, p2_subset)
            bootstrapped_p3 = np.append(bootstrapped_p3, p3_subset)
        # Calculate site pattern counts for the bootstrapped replicate and store the results.
        abba, baba, baaa, abaa = butterfly_site_patterns(
            p1=bootstrapped_p1,
            p2=bootstrapped_p2,
            p3=bootstrapped_p3,
        )
        abba_array = np.append(abba_array, abba)
        baba_array = np.append(baba_array, baba)
        baaa_array = np.append(baaa_array, baaa)
        abaa_array = np.append(abaa_array, abaa)
        abba_hom, baba_hom, baaa_hom, abaa_hom = butterfly_site_patterns(
            p1=bootstrapped_p1,
            p2=bootstrapped_p3,
            p3=bootstrapped_p3,
        )
        abba_hom_array = np.append(abba_hom_array, abba_hom)
        baba_hom_array = np.append(baba_hom_array, baba_hom)
        baaa_hom_array = np.append(baaa_hom_array, baaa_hom)
        abaa_hom_array = np.append(abaa_hom_array, abaa_hom)
    return abba_array, baba_array, baaa_array, abaa_array, abba_hom_array, baba_hom_array, baaa_hom_array, abaa_hom_array

In [44]:
start = time.time()
bs_abba, bs_baba, bs_baaa, bs_abaa, bs_abba_hom, bs_baba_hom, bs_baaa_hom, bs_abaa_hom = butterfly_bootstrap(
    genome_df=der_freq_df,
    p1_id='ag',
    p2_id='ros',
    p3_id='chi',
    n_replicates=1,
)
end = time.time()
print('Run Time: {} minutes'.format(round(((end - start) / 60), 3)))

Run Time: 0.816 minutes


In [45]:
# Print out a summary of each distribution.
print('abba: replicates = {0}; average = {1}'.format(bs_abba.shape[0], round(np.mean(bs_abba), 3)))
print('baba: replicates = {0}; average = {1}'.format(bs_baba.shape[0], round(np.mean(bs_baba), 3)))
print('baaa: replicates = {0}; average = {1}'.format(bs_baaa.shape[0], round(np.mean(bs_baaa), 3)))
print('abaa: replicates = {0}; average = {1}'.format(bs_abaa.shape[0], round(np.mean(bs_abaa), 3)))
print('abba hom: replicates = {0}; average = {1}'.format(bs_abba_hom.shape[0], round(np.mean(bs_abba_hom), 3)))
print('baba hom: replicates = {0}; average = {1}'.format(bs_baba_hom.shape[0], round(np.mean(bs_baba_hom), 3)))
print('baaa hom: replicates = {0}; average = {1}'.format(bs_baaa_hom.shape[0], round(np.mean(bs_baaa_hom), 3)))
print('abaa hom: replicates = {0}; average = {1}'.format(bs_abaa_hom.shape[0], round(np.mean(bs_abaa_hom), 3)))

abba: replicates = 1; average = 50249.869
baba: replicates = 1; average = 17282.744
baaa: replicates = 1; average = 236909.162
abaa: replicates = 1; average = 183353.037
abba hom: replicates = 1; average = 147773.385
baba hom: replicates = 1; average = 11446.041
baaa hom: replicates = 1; average = 271200.443
abaa hom: replicates = 1; average = 171771.35
